
# TradingView Strategy Backtest Playground

Notebook ini memuat pipeline lengkap untuk menguji sinyal strategi yang diekspor dari TradingView. Ia memandu mulai dari memuat data CSV, mengadaptasikannya ke format QF-Lib, menjalankan backtest, hingga analisis trade kalah dan eksperimen optimasi parameter tambahan.



> **Struktur notebook**
>
> 1. Parameter input & pemuatan data
> 2. Adaptasi data ke QF-Lib
> 3. Strategi berbasis sinyal
> 4. Menjalankan backtest & mengekstrak hasil
> 5. Visualisasi
> 6. Analisis trade kalah & investigasi
> 7. Eksplor optimasi parameter
> 8. Dokumentasi & reusable structure


In [1]:
from __future__ import annotations

import sys
from pathlib import Path
from typing import Dict, Iterable

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
plt.switch_backend('Agg')

plt.style.use("seaborn-v0_8-darkgrid")

PROJECT_ROOT = Path('..').resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
if str(PROJECT_ROOT / 'src') not in sys.path:
    sys.path.append(str(PROJECT_ROOT / 'src'))

from src.strategy_backtest import (
    SignalBacktester,
    get_strategy,
    list_strategies,
    load_strategy_csv,
)


## 1. Parameter input & pemuatan data

In [2]:
DATA_FILE = PROJECT_ROOT / 'data' / 'OKX_ETHUSDT.P, 60.csv'
TIME_COLUMN = 'time'
PRICE_COLUMN = 'close'
ASSET_SYMBOL = 'ETHUSDT'

STRATEGY_NAME = 'ema112_atr'
STRATEGY_KWARGS = {
    'slow_span': 112,
    'atr_window': 14,
    'atr_multiplier': 1.5,
    'risk_reward': 2.0,
    'trend_lookback': 5,
}

OPTIMISATION_GRID = {
    'atr_multiplier': [1.0, 1.3, 1.5, 1.8],
    'risk_reward': [1.5, 2.0, 2.5],
}


In [3]:

data, column_mapping = load_strategy_csv(DATA_FILE, time_column=TIME_COLUMN)
print(f'Dataset berisi {len(data):,} bar dengan {len(data.columns)} kolom.')
print('Contoh mapping kolom (sanitised -> original):')
for alias, original in list(column_mapping.items())[:10]:
    print(f'  {alias} -> {original}')

data.head()


Dataset berisi 4,624 bar dengan 41 kolom.
Contoh mapping kolom (sanitised -> original):
  time -> time
  open -> open
  high -> high
  low -> low
  close -> close
  volume -> Volume
  vwap -> VWAP
  vwap_1 -> VWAP.1
  ema -> EMA
  lucid_connector -> LUCID Connector


,open,high,low,close,volume,vwap,vwap_1,ema,lucid_connector,hyperwave,...,atr,histogram,macd,signal,rsi,rsi_minus_based_ma,regular_bullish,regular_bullish_label,regular_bearish,regular_bearish_label
time,,,,,,,,,,,,,,,,,,,,,
2025-05-05 11:00:00,1804.00,1810.20,1798.20,1803.90,2.782500e+08,1807.006655,1807.006655,1821.574753,17891332,46.385720,...,13.590765,0.214971,-4.269723,-4.484694,41.664405,42.233346,NaN,NaN,NaN,NaN
2025-05-05 12:00:00,1803.90,1808.67,1799.01,1804.77,1.762477e+08,1806.857465,1806.857465,1821.277324,1114113,41.013558,...,13.309996,-0.145265,-4.666275,-4.521010,42.248256,42.488844,NaN,NaN,NaN,NaN
2025-05-05 13:00:00,1804.76,1808.50,1793.24,1807.89,4.152894e+08,1806.457798,1806.457798,1821.040380,1114113,39.663401,...,13.449282,-0.123110,-4.674898,-4.551788,44.397488,43.267098,NaN,NaN,NaN,NaN
2025-05-05 14:00:00,1807.89,1814.99,1798.33,1803.61,3.512488e+08,1806.388718,1806.388718,1820.731878,1114113,38.437487,...,13.678619,-0.334413,-4.969803,-4.635391,42.083789,43.392617,NaN,NaN,NaN,NaN
2025-05-05 15:00:00,1803.60,1807.34,1794.40,1800.37,2.290267e+08,1806.090775,1806.090775,1820.371491,1114113,35.028502,...,13.625861,-0.613831,-5.402680,-4.788849,40.368728,44.179336,NaN,NaN,NaN,NaN



## 2. Adaptasi data ke QF-Lib

Data sudah menggunakan `DatetimeIndex` dan kolom harga numerik sehingga siap dipakai sebagai sumber harga untuk QF-Lib. Keluaran sanitasi kolom memudahkan pemanggilan strategi Python di langkah berikutnya.


## 3. Strategi berbasis sinyal

Strategi yang dipakai saat ini adalah **EMA 112 momentum dengan risk-reward 1:2 berbasis ATR**:

- **Entry**: harga penutupan harian menembus EMA 112 dari bawah sementara kemiringan EMA 112 dalam `trend_lookback` hari terakhir positif. Filter slope ini membantu memilah sinyal agar hanya muncul ketika tren benar-benar menguat.
- **Exit**: kombinasi dua kondisi berbasis ATR14 yang adaptif terhadap volatilitas data OKX_BTCUSDT 1D (median ATR ≈ 3.0k, rentang 1.9k–6.5k).
  - *Stop-loss 1R*: keluar ketika penutupan ≤ harga masuk − ATR × multiplier.
  - *Take-profit 2R*: keluar ketika penutupan ≥ harga masuk + risk_reward × ATR × multiplier.
- **Konteks tambahan**: kolom `exit_flag` di sinyal/log trade menandai apakah exit terjadi karena stop atau target sehingga analisis lanjutan lebih mudah.

Strategi tambahan dapat ditaruh dalam folder `src/strategy_backtest/strategies/` dan otomatis terdaftar menggunakan `registry`.


In [4]:

available = list_strategies()
print('Strategi tersedia:', ', '.join(available))
strategy = get_strategy(STRATEGY_NAME, **STRATEGY_KWARGS)
print('Deskripsi strategi:')
print(f"- Nama: {strategy.metadata.name}")
print(f"- Deskripsi: {strategy.metadata.description}")
print(f"- Entry: {strategy.metadata.entry}")
print(f"- Exit: {strategy.metadata.exit}")
print(f"- Parameter default: {strategy.metadata.parameters}")


Strategi tersedia: ema112_atr
Deskripsi strategi:
- Nama: ema112_atr
- Deskripsi: Strategi trend-following berbasis EMA 112 dengan fokus pada momentum harga dan pengelolaan risiko fixed-R multiple menggunakan ATR harian.
- Entry: Entry long ketika harga penutupan menembus ke atas EMA 112 dari posisi di bawahnya dan EMA 112 sedang menanjak dalam jendela lookback yang ditentukan.
- Exit: Exit long ketika harga menutup di bawah level stop berbasis ATR (1R) atau mencapai target keuntungan 2R, menghasilkan rasio risk-reward asimetris 1:2.
- Parameter default: {'slow_span': 112, 'atr_window': 14, 'atr_multiplier': 1.5, 'risk_reward': 2.0, 'trend_lookback': 5}


In [5]:

signals = strategy.generate_signals(data)
print('Kolom sinyal:', list(signals.columns))
signals.head()


Kolom sinyal: ['long_entry', 'long_exit', 'short_entry', 'short_exit', 'ema_trend', 'atr', 'atr_entry', 'active_entry_price', 'stop_level', 'target_level', 'exit_flag']


,long_entry,long_exit,short_entry,short_exit,ema_trend,atr,atr_entry,active_entry_price,stop_level,target_level,exit_flag
time,,,,,,,,,,,
2025-05-05 11:00:00,False,False,False,False,1803.900000,12.000000,NaN,NaN,NaN,NaN,<NA>
2025-05-05 12:00:00,False,False,False,False,1803.915398,10.830000,NaN,NaN,NaN,NaN,<NA>
2025-05-05 13:00:00,False,False,False,False,1803.985745,12.306667,NaN,NaN,NaN,NaN,<NA>
2025-05-05 14:00:00,False,False,False,False,1803.979095,13.395000,NaN,NaN,NaN,NaN,<NA>
2025-05-05 15:00:00,False,False,False,False,1803.915217,13.304000,NaN,NaN,NaN,NaN,<NA>


## 4. Menjalankan backtest & mengekstrak hasil

In [6]:

backtester = SignalBacktester(data=data, price_column=PRICE_COLUMN)
outputs = backtester.run(signals)

metrics = outputs.metrics
trade_summary = outputs.trade_summary
print('Metrik performa (QF-Lib / fallback):')
for key, value in metrics.items():
    if isinstance(value, (int, float, np.floating)):
        print(f'- {key}: {value:.4f}')
    else:
        print(f'- {key}: {value}')

print('Ringkasan trade:')
for key, value in trade_summary.items():
    print(f'- {key}: {value}')


Metrik performa (QF-Lib / fallback):
- total_return: 0.0409
- cagr: 0.0022
- sharpe_ratio: 0.0808
- annualised_vol: 0.0342
- max_drawdown: -0.1195
- avg_drawdown_duration: 183.0000
Ringkasan trade:
- total_trades: 29
- long_trades: 29
- short_trades: 0
- win_rate: 0.41379310344827586
- avg_pnl_pct: 0.0018049397670637129
- median_bars: 16.0


In [7]:
outputs.trades.head()

,trade_id,direction,entry_time,exit_time,entry_price,exit_price,pnl_pct,pnl_currency,bars_held,exit_reason,...,entry_stop_level,entry_target_level,entry_exit_flag,exit_ema_trend,exit_atr,exit_atr_entry,exit_active_entry_price,exit_stop_level,exit_target_level,exit_exit_flag
0,1,Long,2025-05-05 16:00:00,2025-05-06 10:00:00,1807.19,1784.59,-0.012506,-22.60,18,long_exit_signal:stop,...,1787.657500,1846.255000,NaN,1804.863692,12.133571,13.021667,1807.19,1787.657500,1846.255000,stop
1,2,Long,2025-05-15 15:00:00,2025-05-17 00:00:00,2562.66,2500.16,-0.024389,-62.50,33,long_exit_signal:stop,...,2501.212500,2685.555000,NaN,2538.576257,30.068571,40.965000,2562.66,2501.212500,2685.555000,stop
2,3,Long,2025-05-21 19:00:00,2025-05-22 08:00:00,2511.19,2662.40,0.060214,151.21,13,long_exit_signal:target,...,2442.065714,2649.438571,NaN,2525.974763,41.255714,46.082857,2511.19,2442.065714,2649.438571,target
3,4,Long,2025-05-23 13:00:00,2025-05-25 00:00:00,2580.90,2485.61,-0.036921,-95.29,35,long_exit_signal:stop,...,2512.765714,2717.168571,NaN,2562.805725,20.551429,45.422857,2580.90,2512.765714,2717.168571,stop
4,5,Long,2025-06-09 14:00:00,2025-06-09 20:00:00,2529.55,2588.38,0.023257,58.83,6,long_exit_signal:target,...,2500.401786,2587.846429,NaN,2525.514574,22.210714,19.432143,2529.55,2500.401786,2587.846429,target


## 5. Visualisasi

In [8]:
fig, ax = plt.subplots(figsize=(14, 6))
close_prices = data[PRICE_COLUMN]
ax.plot(close_prices.index, close_prices, label='Close', color='C0', linewidth=1.1)

if 'ema_trend' in signals.columns:
    ax.plot(signals.index, signals['ema_trend'], label='EMA 112', color='C1', linewidth=1.2)
if 'stop_level' in signals.columns:
    ax.plot(signals.index, signals['stop_level'], label='Stop 1R', color='C3', linestyle='--', alpha=0.8)
if 'target_level' in signals.columns:
    ax.plot(signals.index, signals['target_level'], label='Target 2R', color='C4', linestyle='--', alpha=0.8)

long_entries = signals['long_entry'].fillna(False)
long_exits = signals['long_exit'].fillna(False)
ax.scatter(close_prices.index[long_entries], close_prices[long_entries], marker='^', color='green', label='Long Entry', zorder=5)

if 'exit_flag' in signals.columns:
    exit_flags = signals['exit_flag'].fillna('')
    stop_mask = long_exits & (exit_flags == 'stop')
    target_mask = long_exits & (exit_flags == 'target')
    remainder = long_exits & ~(stop_mask | target_mask)
    if stop_mask.any():
        ax.scatter(close_prices.index[stop_mask], close_prices[stop_mask], marker='v', color='red', label='Long Exit - Stop', zorder=6)
    if target_mask.any():
        ax.scatter(close_prices.index[target_mask], close_prices[target_mask], marker='v', color='goldenrod', label='Long Exit - Target', zorder=6)
    if remainder.any():
        ax.scatter(close_prices.index[remainder], close_prices[remainder], marker='v', color='red', label='Long Exit', zorder=6)
elif long_exits.any():
    ax.scatter(close_prices.index[long_exits], close_prices[long_exits], marker='v', color='red', label='Long Exit', zorder=6)

ax.set_title(f'{ASSET_SYMBOL} Close dengan Sinyal {STRATEGY_NAME}')
ax.set_ylabel('Harga')
ax.legend()
fig.tight_layout()
display(fig)
plt.close(fig)


<Figure size 1400x600 with 1 Axes>

In [9]:
fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(outputs.results.index, outputs.results['equity_curve'], color='C4', label='Equity Curve')
ax.set_title('Equity Curve Strategi')
ax.set_ylabel('Notional')
ax.legend()
fig.tight_layout()
display(fig)
plt.close(fig)

fig, ax = plt.subplots(figsize=(10, 4))
ax.hist(outputs.trades['pnl_pct'], bins=20, color='C0', alpha=0.7)
ax.set_title('Distribusi PnL per Trade')
ax.set_xlabel('PnL %')
ax.set_ylabel('Frekuensi')
fig.tight_layout()
display(fig)
plt.close(fig)


<Figure size 1400x400 with 1 Axes>

<Figure size 1000x400 with 1 Axes>

## 6. Analisis trade kalah & investigasi

In [10]:

losing_trades = outputs.trades[outputs.trades['pnl_pct'] < 0]
print(f'Jumlah trade kalah: {len(losing_trades)} dari total {len(outputs.trades)}')
if not losing_trades.empty:
    print('Statistik ringkas trade kalah:')
    display(losing_trades[['pnl_pct', 'bars_held', 'exit_reason']].describe(include='all'))

    grouped = losing_trades.groupby('exit_reason')['pnl_pct'].agg(['count', 'mean', 'min'])
    print('Kinerja berdasarkan alasan exit:')
    display(grouped)

    context_cols = [col for col in outputs.trades.columns if col.startswith('entry_') or col.startswith('exit_')]
    if context_cols:
        print('Contoh konteks indikator pada trade kalah:')
        display(losing_trades[['trade_id', 'direction'] + context_cols].head())
else:
    print('Tidak ada trade kalah pada konfigurasi ini.')


Jumlah trade kalah: 17 dari total 29
Statistik ringkas trade kalah:


,pnl_pct,bars_held,exit_reason
count,17.000000,17.000000,17
unique,NaN,NaN,1
top,NaN,NaN,long_exit_signal:stop
freq,NaN,NaN,17
mean,-0.021298,18.647059,NaN
std,0.007841,16.658111,NaN
min,-0.036921,3.000000,NaN
25%,-0.026020,7.000000,NaN
50%,-0.021972,16.000000,NaN
75%,-0.014407,21.000000,NaN


Kinerja berdasarkan alasan exit:


,count,mean,min
exit_reason,,,
long_exit_signal:stop,17,-0.021298,-0.036921


Contoh konteks indikator pada trade kalah:


,trade_id,direction,entry_time,exit_time,entry_price,exit_price,exit_reason,entry_ema_trend,entry_atr,entry_atr_entry,...,entry_stop_level,entry_target_level,entry_exit_flag,exit_ema_trend,exit_atr,exit_atr_entry,exit_active_entry_price,exit_stop_level,exit_target_level,exit_exit_flag
0,1,Long,2025-05-05 16:00:00,2025-05-06 10:00:00,1807.19,1784.59,long_exit_signal:stop,1803.973178,13.021667,13.021667,...,1787.657500,1846.255000,NaN,1804.863692,12.133571,13.021667,1807.19,1787.657500,1846.255000,stop
1,2,Long,2025-05-15 15:00:00,2025-05-17 00:00:00,2562.66,2500.16,long_exit_signal:stop,2511.567156,40.965000,40.965000,...,2501.212500,2685.555000,NaN,2538.576257,30.068571,40.965000,2562.66,2501.212500,2685.555000,stop
3,4,Long,2025-05-23 13:00:00,2025-05-25 00:00:00,2580.90,2485.61,long_exit_signal:stop,2576.974483,45.422857,45.422857,...,2512.765714,2717.168571,NaN,2562.805725,20.551429,45.422857,2580.90,2512.765714,2717.168571,stop
5,6,Long,2025-06-16 05:00:00,2025-06-16 22:00:00,2606.35,2568.80,long_exit_signal:stop,2575.169082,24.600000,24.600000,...,2569.450000,2680.150000,NaN,2589.549314,30.767143,24.600000,2606.35,2569.450000,2680.150000,stop
7,8,Long,2025-06-30 15:00:00,2025-07-01 12:00:00,2473.40,2436.37,long_exit_signal:stop,2441.402185,21.897143,21.897143,...,2440.554286,2539.091429,NaN,2451.083154,15.447143,21.897143,2473.40,2440.554286,2539.091429,stop


## 7. Eksplor optimasi parameter

In [11]:

def sweep_parameters(base_kwargs: Dict[str, float], grid: Dict[str, Iterable[float]]):
    if not grid:
        return []
    results = []
    for multiplier in grid.get('atr_multiplier', []):
        kwargs = dict(base_kwargs)
        kwargs['atr_multiplier'] = multiplier
        strat = get_strategy(STRATEGY_NAME, **kwargs)
        sig = strat.generate_signals(data)
        backtest = backtester.run(sig)
        results.append({
            'atr_multiplier': multiplier,
            'total_return': backtest.metrics.get('total_return', np.nan),
            'sharpe_ratio': backtest.metrics.get('sharpe_ratio', np.nan),
            'trades': backtest.trade_summary['total_trades'],
        })
    return results

opt_results = sweep_parameters(STRATEGY_KWARGS, OPTIMISATION_GRID)
if opt_results:
    optimisation_df = pd.DataFrame(opt_results).sort_values('sharpe_ratio', ascending=False)
    display(optimisation_df)
else:
    print('Grid optimasi kosong – lewati tahap ini.')


,atr_multiplier,total_return,sharpe_ratio,trades
0,1.0,0.046736,0.107598,33
2,1.5,0.040873,0.080849,29
3,1.8,0.008135,0.030549,26
1,1.3,-0.019326,-0.017800,30



## 8. Dokumentasi & reusable structure

- Ganti `DATA_FILE` untuk menguji dataset TradingView lain yang memakai format serupa.
- Tambah atau modifikasi strategi di `src/strategy_backtest/strategies/`, lalu daftarkan nama modul di `registry.py`.
- Notebook ini mengandalkan `SignalBacktester` untuk menghasilkan trade log, metrik QF-Lib, serta visualisasi standar sehingga langkah analisis tetap konsisten antar strategi.
- Simpan variasi eksperimen (mis. hasil optimasi) ke file CSV tambahan bila diperlukan untuk laporan lebih lanjut.
